###Ingestion de la carpeta "movie_language" con archivos .json

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

####Paso 1- Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
movie_language_schema = StructType(fields=[
    StructField("movieId", IntegerType(), True),
    StructField("languageId", IntegerType(), True),
    StructField("languageRoleId", IntegerType(), True)
])

In [0]:
movie_language_df = spark.read \
                .schema(movie_language_schema) \
                .option("multiLine", True) \
                .json(f"{bronze_folder_path}/{v_file_date}/movie_language")
#                .csv("/mnt/mymoviehistory/bronze/production_company/production_company_*.csv")                

In [0]:
display(movie_language_df)

In [0]:
movie_language_df.count()
#para ver todos los registros que se han ingestado de todos los archivos

####Paso 2 - Renombrar las columnas y añadir nuevas columnas
- "movieId" renombar a "movie_id", "languageId" a "language_id"
- Agregar las columnas "ingestion_date" y "environment"

In [0]:
from pyspark.sql.functions import col, current_timestamp, lit

In [0]:
movie_language_final_df = add_ingestion_dat(movie_language_df) \
                            .withColumnsRenamed({"movieId":"movie_id",
                                                  "languageId":"language_id"}) \
                            .withColumn("environment", lit(v_environment)) \
                            .withColumn("file_date",lit(v_file_date))
display(movie_language_final_df)

####Paso 3 - Elimina la columna "languageRoleId"

In [0]:
movie_language_final2_df=movie_language_final_df.drop(col("languageRoleId"))

####Paso 4 - Escribir la salida en un formato "Parquet"


In [0]:
#movie_language_final2_df.write.mode("overwrite").parquet(f"{silver_folder_path}/movies_languages")
#movie_language_final2_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.movies_languages")
merge_condition='tgt.movie_id = src.movie_id and tgt.language_id = src.language_id and tgt.file_date = src.file_date'
merge_delta_lake(movie_language_final2_df, "movie_silver", "movies_languages", silver_folder_path, merge_condition, "file_date")

In [0]:
movie_language_final2_df.write.mode("append").partitionBy("file_date").format("delta").save(f"{silver_folder_path}/movies_languages")

In [0]:
%sql
select file_date, count(1) 
from movie_silver.movies_languages
group by file_date;

In [0]:
display(spark.read.format("delta").load("/mnt/mymoviehistory/silver/movies_languages"))

In [0]:
dbutils.notebook.exit("Success")